# Aligning HST ACS/HRC Images Using `tweakwcs`

***
## About this Notebook
**Author:** Mihai Cara, STScI
<br>**Initial version on:** 11/20/2018
<br>**Updated on:** 02/07/2019

***
## Introduction

Often the World Coordinate System (WCS) of images may contain small errors. These alignment errors in the WCS of the images need to be removed before images can be further processed, e.g., before they can be combined into a mosaiced image. The images are said to be aligned (in a relative sense) _on the sky_ when image coordinates _of the same object_ (present in several images) can be converted aproximately the same sky coordinates (using appropriate image's WCS).

In this notebook we illustrate how to create source catalogs using `photutils` package and then how to match sources from image catalogs and find aligned `WCS` using `tweakwcs` package.

***
## Imports

In [ ]:
import shutil
import glob
import os
import sys
import logging

import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.nddata import NDData
from astroquery.mast import Observations
from photutils import detect_threshold, DAOStarFinder

from stwcs.wcsutil import HSTWCS
from drizzlepac import updatehdr

from tweakwcs import fit_wcs, align_wcs, FITSWCS, TPMatch, WCSImageCatalog

***
## Setup Logging

Add basic support for loging to `stdout`:

In [ ]:
logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
logger.addHandler(handler)

***
## Download Data

For this example, we have chosen HST ACS/WFC observation of NGC104 in the F606W filter. The data come from the SM3/ACS proposal 9019 _"HRC flat field stability"_ (PI: Ralph Bohlin).

Data are downloaded using the `astroquery` API to access the [MAST](http://archive.stsci.edu) archive. The `astroquery.mast` [documentation](http://astroquery.readthedocs.io/en/latest/mast/mast.html) has more examples for how to find and download data from MAST.

In [ ]:
# If mastDownload directory already exists, delete it
# and all subdirectories it contains:
if os.path.isdir('mastDownload'):
    shutil.rmtree('mastDownload')

# Retrieve the observation information.
obs_table = Observations.query_criteria(obs_id='j8bt06*', filters='F606W', obstype='ALL')
products = Observations.get_product_list(obs_table)

# Download only the 'j8bt06nyq' and 'j8bt06nzq' images:
Observations.download_products(products, mrp_only=False, obs_id=['j8bt06nyq', 'j8bt06nzq'],
                               productSubGroupDescription=['FLC', 'FLT'], 
                               extension='fits')

def copy_mast_to_cwd():
    """
    Move the files from the mastDownload directory to the current working
    directory and make a backup of the files. Return a list of image file
    names in the CWD.
    
    """
    downloaded_fits_files = glob.glob('mastDownload/HST/j*/j*flt.fits')
    fits_files = []
    for fil in downloaded_fits_files:
        base_name = os.path.basename(fil)
        fits_files.append(base_name)
        if os.path.isfile(base_name):
            os.remove(base_name)
        shutil.copy2(fil, '.')
        
    return fits_files

fits_files = copy_mast_to_cwd()

***
## EXAMPLE 1: Simple Workflow to Align Two or More Images

In this example we illustrate the use of convenience function `align_wcs()` to align two images downloaded in the previous step. 

### 1.1 Create Source Catalogs and FITSWCS WCS Corrector Objects

This stage fundamentally consists of three steps:
- create source catalogs for each image whose WCS needs to be aligned;
- create a tangent-plane WCS corrector object specific for image's WCS and supply the catalog to this object through it's `meta` attribute, essetially creating a WCS-catalog combination;
- call `align_wcs` to align WCS.

For the first step, we use `photutils` package to find stars in the images. One can use any other tools star finding. Then, for the second step, we create FITSWCS corrector object which is appropriate for FITS WCS used in HST images. 

In [ ]:
wcs_catalogs = []
for group_id, file in enumerate(fits_files):
    group_id += 1
    with fits.open(file) as hdulist:
        # extract data and WCS from each SCI extension:
        im_data = hdulist[('SCI', 1)].data
        w = HSTWCS(hdulist, ('SCI', 1))
        
        # find sources / create catalogs
        threshold = detect_threshold(im_data, snr=100.0)[0, 0]
        daofind = DAOStarFinder(fwhm=2.5, threshold=threshold, exclude_border=True)
        cat = daofind(im_data)
        cat.rename_column('xcentroid', 'x')
        cat.rename_column('ycentroid', 'y')
        print("Length of catalog #{:d}: {:d}".format(group_id, len(cat)))

        # create tangent-plane WCS corrector:
        wcscat = FITSWCS(wcs=w,
                         meta={
                             'group_id': group_id,
                             'name': 'im{:d} sources'.format(group_id),  # or use any other naming scheme
                             'catalog': cat,
                             'filename': file,  # optional info
                             'ext': ('SCI', 1),  # optional info
                         }
                        )
        wcs_catalogs.append(wcscat)

### 1.2 Align Images (Find Corrected WCS):

In [ ]:
align_wcs(wcs_catalogs, expand_refcat=True, enforce_user_order=False);

### 1.3 Update FITS File Headers with Aligned WCS

In [ ]:
for file, w in zip(fits_files, [im.wcs for im in wcs_catalogs]):
    with fits.open(file, mode='update') as hdulist:
        updatehdr.update_wcs(hdulist, 1, w, wcsname='TWEAK', reusename=True, verbose=False)

The above code serves as a simple illustration of updating files. The logic could be expanded to utilize the information stored in the `meta` dictionary such as file name and extension.

***
## EXAMPLE 2: Customizable Workflow to Align Two or More Images or to Align to an External Reference Catalog

In this example we show how to use lower-level functions to align two images. This approach allows significantly higher customization compared to the use of the convenience function `align_wcs()` from Example 1. In addition, this approach allows inspection and logging of intermediate results such as number of matched sources, their indices in the corresponding catalogs, linear fit results, fit residuals, etc.

### 2.1 Get a Fresh Copy of Data

In [ ]:
fits_files = copy_mast_to_cwd()

### 2.2 Create Catalogs and Create a Telescope/Instrument-specific "corrector" WCS object

Below we take the sources from the first image to create a "reference" catalog. Therefore this example can be used also for aligning images to _external_ "reference" catalogs. Since we are working with HST images that use FITS WCS, we will use `FITSWCS` tangent plane corrector specific to FITS WCS.

In [ ]:
catalogs = []

for group_id, file in enumerate(fits_files):
    group_id += 1
    with fits.open(file) as hdulist:
        im_data = hdulist[('SCI', 1)].data
        dq_data = hdulist[('DQ', 1)].data
        w = HSTWCS(hdulist, ('SCI', 1))
        
        # create FITS WCS corrector object
        wcs_corrector = FITSWCS(w)
        
        # find stars:
        threshold = detect_threshold(im_data, snr=100.0)[0, 0]
        daofind = DAOStarFinder(fwhm=2.5, threshold=threshold, exclude_border=True)
        cat = daofind(im_data)
        cat.rename_column('xcentroid', 'x')
        cat.rename_column('ycentroid', 'y')
        cat.meta['name'] = 'im{:d} sources'.format(group_id)
        cat.meta['file_name'] = file
        print("Length of catalog #{:d}: {:d}".format(group_id, len(cat)))
        
        catalogs.append((cat, wcs_corrector))

Create a "reference" catalog based on the first image's stars. A reference catalog must have star coordinates in world coordinates. When using external reference catalog, this step essentially can be skipped.

In [ ]:
refcat, refwcs = catalogs.pop(0)
refcat.meta['name'] = 'REFCAT ({})'.format(refcat.meta['name'])

# convert image coordinates to sky coords:
ra, dec = refwcs.det_to_world(refcat['x'], refcat['y'])
refcat['RA'] = ra
refcat['DEC'] = dec

### 2.3 Match Catalogs and Align Image WCS

In [ ]:
match = TPMatch(searchrad=5, separation=0.1, tolerance=5, use2dhist=False)

for imcat, imwcs in catalogs:
    # Match sources in the catalogs:
    ridx, iidx = match(refcat, imcat, imwcs)
    
    # Align image WCS:
    aligned_imwcs = fit_wcs(refcat[ridx], imcat[iidx], imwcs).wcs
    imcat.meta['aligned_wcs'] = aligned_imwcs

### 2.4 Update FITS File Headers with Aligned WCS

In [ ]:
for cat, _ in catalogs:
    with fits.open(cat.meta['file_name'], mode='update') as hdulist:
        updatehdr.update_wcs(hdulist, 1, cat.meta['aligned_wcs'], wcsname='TWEAK', reusename=True, verbose=False)

***
## Delete Downloaded Data

In [ ]:
# Delete the mastDownload directory and all subdirectories it contains:
if os.path.isdir('mastDownload'):
    shutil.rmtree('mastDownload')